# This notebook is a PoC attempt on memory usage optimization in Pandas

### Contents:

#####      1. Optimization directly on pandas dataframes
#####      2. Using NumPy arrays vs Pandas DataFrames
#####      3. Dask DataFrames vs Pandas DataFrames

## 1. Optimization directly on pandas dataframes

   #### a. Alter the column dtypes:
    
  When we create a Pandas DataFrame, Pandas will assign the highest memory datatype to columns by default. For example, when     it detects a column of integers it will assign ```int64``` to the dtype of the column, regardless of the size of the integer   values. This consumes a lot of unnecessary memory. We can use the ```astype()``` method to downgrade the datatypes. Here is     an example:

In [141]:
# Imports

import pandas as pd
import numpy as np
import os

In [142]:
# A Method to generate dummy dataframe that we will use throughout the notebook.

def generate_fake_dataframe(size, cols, col_names = None, intervals = None, seed = None):
    
    categories_dict = {'animals': ['cow', 'rabbit', 'duck', 'shrimp', 'pig', 'goat', 'crab', 'deer', 'bee', 'sheep', 'fish', 'turkey', 'dove', 'chicken', 'horse'],
                       'names'  : ['James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth', 'Ahmed', 'Barbara', 'Richard', 'Susan', 'Salomon', 'Juan Luis'],
                       'cities' : ['Stockholm', 'Denver', 'Moscow', 'Marseille', 'Palermo', 'Tokyo', 'Lisbon', 'Oslo', 'Nairobi', 'Río de Janeiro', 'Berlin', 'Bogotá', 'Manila', 'Madrid', 'Milwaukee'],
                       'colors' : ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'black', 'white']
                      }
    default_intervals = {"i" : (0,10), "f" : (0,100), "c" : ("names", 5), "d" : ("2020-01-01","2020-12-31")}
    rng = np.random.default_rng(seed)

    first_c = default_intervals["c"][0]
    categories_names = cycle([first_c] + [c for c in categories_dict.keys() if c != first_c])
    default_intervals["c"] = (categories_names, default_intervals["c"][1])
    
    if isinstance(col_names,list):
        assert len(col_names) == len(cols), f"The fake DataFrame should have {len(cols)} columns but col_names is a list with {len(col_names)} elements"
    elif col_names is None:
        suffix = {"c" : "cat", "i" : "int", "f" : "float", "d" : "date"}
        col_names = [f"column_{str(i)}_{suffix.get(col)}" for i, col in enumerate(cols)]

    if isinstance(intervals,list):
        assert len(intervals) == len(cols), f"The fake DataFrame should have {len(cols)} columns but intervals is a list with {len(intervals)} elements"
    else:
        if isinstance(intervals,dict):
            assert len(set(intervals.keys()) - set(default_intervals.keys())) == 0, f"The intervals parameter has invalid keys"
            default_intervals.update(intervals)
        intervals = [default_intervals[col] for col in cols]
    df = pd.DataFrame()
    for col, col_name, interval in zip(cols, col_names, intervals):
        if interval is None:
            interval = default_intervals[col]
        assert (len(interval) == 2 and isinstance(interval, tuple)) or isinstance(interval, list), f"This interval {interval} is neither a tuple of two elements nor a list of strings."
        if col in ("i","f","d"):
            start, end = interval
        if col == "i":
            df[col_name] = rng.integers(start, end, size)
        elif col == "f":
            df[col_name] = rng.uniform(start, end, size)
        elif col == "c":
            if isinstance(interval, list):
                categories = np.array(interval)
            else:
                cat_family, length = interval
                if isinstance(cat_family, cycle):
                    cat_family = next(cat_family)
                assert cat_family in categories_dict.keys(), f"There are no samples for category '{cat_family}'. Consider passing a list of samples or use one of the available categories: {categories_dict.keys()}"
                categories = rng.choice(categories_dict[cat_family], length, replace = False, shuffle = True)
            df[col_name] = rng.choice(categories, size, shuffle = True)
        elif col == "d":
            df[col_name] = rng.choice(pd.date_range(start, end), size)
    return df 

__This method is fetched from the article [Generating fake data with pandas, very quickly](https://towardsdatascience.com/generating-fake-data-with-pandas-very-quickly-b99467d4c618)__

In [143]:
# Generating a dummy data frame with 5000000 rows and 10 columns.
# 'c' = category, 'i' = integer, 'f' = float, 'd' = datetime.

dummy_df = generate_fake_dataframe(size = 5000000, cols =  "cififdiccd")

In [144]:
# Info about the data frame
dummy_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   column_0_cat    object        
 1   column_1_int    int64         
 2   column_2_float  float64       
 3   column_3_int    int64         
 4   column_4_float  float64       
 5   column_5_date   datetime64[ns]
 6   column_6_int    int64         
 7   column_7_cat    object        
 8   column_8_cat    object        
 9   column_9_date   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(3), object(3)
memory usage: 1.2 GB


As I have mentioned before, every datatype is the largest type as possible. The memory usage is over 1 GB

In [145]:
# Saving the data frame as csv
df_as_csv = dummy_df.to_csv('out.csv')

This might take a while.

In [146]:
# Path of the csv
data_dir = '.\out.csv'

In [147]:
# Read the csv
df = pd.read_csv(data_dir)

In [148]:
# Df info
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Unnamed: 0      int64  
 1   column_0_cat    object 
 2   column_1_int    int64  
 3   column_2_float  float64
 4   column_3_int    int64  
 5   column_4_float  float64
 6   column_5_date   object 
 7   column_6_int    int64  
 8   column_7_cat    object 
 9   column_8_cat    object 
 10  column_9_date   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1.8 GB


Notice that the datetime columns are now type of object. Now it consumes even more memory.

In [149]:
# Memory usage of each column
df.memory_usage(index = True, deep = True)

Index                   128
Unnamed: 0         40000000
column_0_cat      318994442
column_1_int       40000000
column_2_float     40000000
column_3_int       40000000
column_4_float     40000000
column_5_date     335000000
column_6_int       40000000
column_7_cat      308996545
column_8_cat      348020080
column_9_date     335000000
dtype: int64

In [150]:
# Overall memory consumption
memory_consumption = df.memory_usage(index = True, deep = True).sum() / 1024**2
print(f"Overall memory consumption: {memory_consumption} MB")

Overall memory consumption: 1798.6404371261597 MB


#### In order to save some memory, we can inspect the columns and alter the datatypes

 Take the "column_1_int" column. It is of type ```int64```. Let's check out the max and min values and see if they 
 span the scope of the datatype.

In [151]:
print("Data type of column_1_int column is", df.column_1_int.dtype)
print("Maximum value in column_1_int column is", df.column_1_int.max())
print("Minimum value in column_1_int column is", df.column_1_int.min())

Data type of column_1_int column is int64
Maximum value in column_1_int column is 9
Minimum value in column_1_int column is 0


As we can see the column has only integers between 0-10 but the datatype is still ```int64```. Now let's cast the dtype to ````int8```` and see the difference:

In [152]:
print("Memory usage before changing the datatype:", df.column_1_int.memory_usage(deep = True))

df["column_1_int"] = df.column_1_int.astype(np.int8)

print("Memory usage after changing the datatype:", df.column_1_int.memory_usage(deep = True))

# It has reduced the memory usage almost by 88%

Memory usage before changing the datatype: 40000128
Memory usage after changing the datatype: 5000128


__It has reduced the memory usage almost by 88%__

 Take the "column_2_float" column. It is of type ``float64``. Let's check out the max and min values and see if they 
 span the scope of the datatype.

In [153]:
print("Data type of column_2_float column is", df.column_2_float.dtype)
print("Maximum value in column_2_float column is", df.column_2_float.max())
print("Minimum value in column_2_float column is", df.column_2_float.min())

Data type of column_2_float column is float64
Maximum value in column_2_float column is 99.99999248133842
Minimum value in column_2_float column is 9.635407993702216e-06


As we can see the column has only floating point numbers between 0-10 but the datatype is still ```float64```. Now let's cast the dtype to ````float16```` and see the difference:

In [154]:
print("Memory usage before changing the datatype:", df.column_2_float.memory_usage(deep = True))

df["column_2_float"] = df.column_2_float.astype(np.float16)

print("Memory usage after changing the datatype:", df.column_2_float.memory_usage(deep = True))

# It has reduced the memory usage almost by 75%

Memory usage before changing the datatype: 40000128
Memory usage after changing the datatype: 10000128


#### Pandas assign "object" for columns with categorical data. 
#### If we have a column of type "object" and has few unique values, we can alter the dtype to "categorical"

In [155]:
# Let's see the number of unique values in 'column_5_date' column

print("Data type of column_5_date column is", df.column_5_date.dtype)
print("Number of Unique values in column_5_date column is", df.column_5_date.nunique())
print("The number of rows", df.shape[0])

Data type of column_5_date column is object
Number of Unique values in column_5_date column is 366
The number of rows 5000000


In [156]:
# We have 5000000 values in the 'column_5_date' column but only 366 of them are unique.
# It means we can represent this column as 'categorical'

print("Memory usage before changing the datatype:", df.column_5_date.memory_usage())

df["column_5_date"] = df.column_5_date.astype("category")

print("Memory usage after changing the datatype:", df.column_5_date.memory_usage())

Memory usage before changing the datatype: 40000128
Memory usage after changing the datatype: 10011352


#### There are some considerations to keep in mind when representing datetime values as categorical
**Loss of precision:** Categorical data type represents data using a numerical code associated with each unique category. By converting datetime values to categorical, you may lose the precision of the original timestamps. If you require high precision for your datetime calculations, it's better to use the 'datetime' data type.

**Limited functionality:** Categorical data type provides benefits in terms of memory optimization and faster operations on the column. However, some datetime-specific operations and functions may not be available or may behave differently when applied to categorical datetime values. If your analysis heavily relies on datetime functionality, it's advisable to keep the column as 'datetime' type.

Here is a util that does the downcasting for you. Also when the majority of the values are missing in a column. It keeps that column in a Sparse Array, which saves up some memory

In [173]:
SPARSITY_THRESHOLD = 0.5

def optimize(df: pd.DataFrame) -> pd.DataFrame:
    float_types = {np.finfo(np.float16).max: np.float16, np.finfo(np.float32).max: np.float32}
    int_types = {np.iinfo(np.int8).max: np.int8, np.iinfo(np.int16).max: np.int16, np.iinfo(np.int32).max: np.int32}
    
    for dtype in ['float64', 'int64']:
        selected_columns = df.select_dtypes(include=[dtype])
        max_values = selected_columns.max()
        types = float_types if dtype == 'float64' else int_types

        for col in selected_columns.columns:
            if df[col].isna().sum() / len(df[col]) > SPARSITY_THRESHOLD:  # If mostly NaN values
                df[col] = pd.arrays.SparseArray(df[col])
            else:
                max_val = max_values[col]
                if not pd.isna(max_val):  # skip columns with only NaNs
                    for max_type_val, type_val in types.items():
                        if max_val <= max_type_val:
                            df[col] = df[col].astype(type_val)
                            break

    for col in df.select_dtypes(include=['object']).columns:
        if df[col].isna().sum() / len(df[col]) > SPARSITY_THRESHOLD:  # If mostly NaN values
            df[col] = pd.arrays.SparseArray(df[col])
        else:
            try:
                df[col] = pd.to_datetime(df[col])
            except ValueError:
                num_unique_values = len(df[col].unique())
                num_total_values = len(df[col])
                if num_unique_values / num_total_values < SPARSITY_THRESHOLD:
                    df[col] = df[col].astype('category')

    return df


__Here is a demonstration of the ``optimize()`` method__

In [159]:
# Trying out the optimize() method

df_temp = pd.read_csv(data_dir)
print(df_temp.info(memory_usage = "deep"))
print("***********************")
print("Memory usage of df before optimization: {:.2f} MB".format(df_temp.memory_usage(deep=True).sum() / 1024**2))

print("**************************************************")

df_temp = optimize(df_temp)
print(df_temp.info(memory_usage = "deep"))
print("***********************")
print("Memory usage of df before optimization: {:.2f} MB".format(df_temp.memory_usage(deep=True).sum() / 1024**2))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Unnamed: 0      int64  
 1   column_0_cat    object 
 2   column_1_int    int64  
 3   column_2_float  float64
 4   column_3_int    int64  
 5   column_4_float  float64
 6   column_5_date   object 
 7   column_6_int    int64  
 8   column_7_cat    object 
 9   column_8_cat    object 
 10  column_9_date   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1.8 GB
None
***********************
Memory usage of df before optimization: 1798.64 MB
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 11 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Unnamed: 0      int32         
 1   column_0_cat    category      
 2   column_1_int    int8          
 3   column_2_float  floa

__Simulating a case:__ Merging the optimized dataframe with another dataframe that has an integer value larger than ``int8`` under a column with the same name.

In [164]:
# Generating a dummy data frame with 5000 rows and 10 columns. Merging two dataframes with 5 million rows will exceed the memory
# constraints.

dummy_df_2 = generate_fake_dataframe(size = 5000, cols =  "cififdiccd")
df_optimized = dummy_df_2
dummy_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   column_0_cat    5000 non-null   object        
 1   column_1_int    5000 non-null   int64         
 2   column_2_float  5000 non-null   float64       
 3   column_3_int    5000 non-null   int64         
 4   column_4_float  5000 non-null   float64       
 5   column_5_date   5000 non-null   datetime64[ns]
 6   column_6_int    5000 non-null   int64         
 7   column_7_cat    5000 non-null   object        
 8   column_8_cat    5000 non-null   object        
 9   column_9_date   5000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(3), object(3)
memory usage: 390.8+ KB


In [166]:
df_optimized = optimize(df_optimized)
df_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   column_0_cat    5000 non-null   category      
 1   column_1_int    5000 non-null   int32         
 2   column_2_float  5000 non-null   float16       
 3   column_3_int    5000 non-null   int8          
 4   column_4_float  5000 non-null   float16       
 5   column_5_date   5000 non-null   datetime64[ns]
 6   column_6_int    5000 non-null   int8          
 7   column_7_cat    5000 non-null   category      
 8   column_8_cat    5000 non-null   category      
 9   column_9_date   5000 non-null   datetime64[ns]
dtypes: category(3), datetime64[ns](2), float16(2), int32(1), int8(2)
memory usage: 142.3 KB


In [167]:
# Inserting a large integer number

df_to_merge = dummy_df_2
df_to_merge.at[2, 'column_1_int'] = 10000000
df_to_merge.head()

,column_0_cat,column_1_int,column_2_float,column_3_int,column_4_float,column_5_date,column_6_int,column_7_cat,column_8_cat,column_9_date
0,Patricia,4,24.7500,4,57.000000,2020-08-05,1,horse,Marseille,2020-04-27
1,Michael,0,79.0625,9,95.125000,2020-04-04,5,pig,Nairobi,2020-08-19
2,Michael,10000000,78.4375,6,1.405273,2020-11-13,8,pig,Lisbon,2020-02-09
3,Susan,4,79.3125,1,85.000000,2020-09-10,7,cow,Bogotá,2020-06-22
4,Michael,4,85.7500,9,72.562500,2020-06-13,4,fish,Bogotá,2020-02-03


In [168]:
# Performing a left join
merged_df = pd.merge(df_to_merge, df_optimized, on='column_1_int', how='left')

In [169]:
merged_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2502224 entries, 0 to 2502223
Data columns (total 19 columns):
 #   Column            Dtype         
---  ------            -----         
 0   column_0_cat_x    category      
 1   column_1_int      int32         
 2   column_2_float_x  float16       
 3   column_3_int_x    int8          
 4   column_4_float_x  float16       
 5   column_5_date_x   datetime64[ns]
 6   column_6_int_x    int8          
 7   column_7_cat_x    category      
 8   column_8_cat_x    category      
 9   column_9_date_x   datetime64[ns]
 10  column_0_cat_y    category      
 11  column_2_float_y  float16       
 12  column_3_int_y    int8          
 13  column_4_float_y  float16       
 14  column_5_date_y   datetime64[ns]
 15  column_6_int_y    int8          
 16  column_7_cat_y    category      
 17  column_8_cat_y    category      
 18  column_9_date_y   datetime64[ns]
dtypes: category(6), datetime64[ns](4), float16(4), int32(1), int8(4)
memory usage: 148.0

As we can see the datatypes are back to being unefficient. Optimizing again:

In [170]:
merged_df = optimize(merged_df)

In [171]:
merged_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2502224 entries, 0 to 2502223
Data columns (total 19 columns):
 #   Column            Dtype         
---  ------            -----         
 0   column_0_cat_x    category      
 1   column_1_int      int32         
 2   column_2_float_x  float16       
 3   column_3_int_x    int8          
 4   column_4_float_x  float16       
 5   column_5_date_x   datetime64[ns]
 6   column_6_int_x    int8          
 7   column_7_cat_x    category      
 8   column_8_cat_x    category      
 9   column_9_date_x   datetime64[ns]
 10  column_0_cat_y    category      
 11  column_2_float_y  float16       
 12  column_3_int_y    int8          
 13  column_4_float_y  float16       
 14  column_5_date_y   datetime64[ns]
 15  column_6_int_y    int8          
 16  column_7_cat_y    category      
 17  column_8_cat_y    category      
 18  column_9_date_y   datetime64[ns]
dtypes: category(6), datetime64[ns](4), float16(4), int32(1), int8(4)
memory usage: 148.0

#### b. Loading the data in chunks
    
  Loading the data in chunks could be useful when dealing with large datasets. By loading data in smaller portions, or           "chunks", memory usage is kept to a minimum, preventing potential slowdowns or crashes that could occur if the system runs     out of memory. This could be especially beneficial in environments where memory resources are limited. Also it gives the       opportunity to process the chunks independently. Hence, more flexible data processing is possible.
  
  However processing the data in chunks might end up in increased total processing time due to repeated disk operations.

##### ``fetchall()`` vs ``read_sql()``
   The both functions load the data into memory at once. Pandas' ``read_sql()`` loads the data as a DataFrame, ``fetchall()``      of ``pyodbc`` or similar libraries load the data as a list of tuples. ``fetchall()`` could potentially use more memory when    the data contains columns with mixed types.

In [126]:
# function to get current memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024**2  # return memory usage in MB

The ``get_memory_usage`` function returns the Resident Set Size (RSS) of the current process. RSS is the portion of the process's memory that is held in RAM.

When a process is started, the operating system allocates a certain amount of physical memory (RAM) for it. This memory space is divided into several segments, each with a specific purpose:

The RSS value refers to the portion of this memory which is in RAM, i.e., it includes the size of the stack, the heap, and the data segment of the process. It excludes memory that is swapped out to disk or memory-mapped files.

In [125]:
import pyodbc
import psutil

# connect to the SQL Server database
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=AdventureWorks2019;'
    r'Trusted_Connection=yes;'
)
cnxn = pyodbc.connect(conn_str)

# define SQL query
query1 = "SELECT * FROM Sales.SalesOrderDetail"

# specify chunk size
chunk_size = 50000

# initialize an empty list to store chunks
chunks = []

print(f"Memory usage before loading data: {get_memory_usage():.2f} MB")

# read and process data in chunks
chunk_number = 0
for chunk in pd.read_sql(query1, cnxn, chunksize=chunk_size):
    # print memory usage for each chunk
    chunk_number += 1
    print(f"Memory usage after loading chunk {chunk_number}: {get_memory_usage():.2f} MB")
    print(f"Chunk {chunk_number} size: {chunk.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # process each chunk as a separate dataframe if needed
    # here we simply add it to the list
    chunks.append(chunk)

# Combine all chunks into one DataFrame
df1 = pd.concat(chunks, ignore_index=True)

print(f"Memory usage after combining all chunks: {get_memory_usage():.2f} MB")
print(f"Final DataFrame size: {df1.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


Memory usage before loading data: 1466.52 MB


C:\Users\harun\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Memory usage after loading chunk 1: 1525.12 MB
Chunk 1 size: 10.70 MB
Memory usage after loading chunk 2: 1539.97 MB
Chunk 2 size: 9.69 MB
Memory usage after loading chunk 3: 1534.78 MB
Chunk 3 size: 4.09 MB
Memory usage after combining all chunks: 1537.83 MB
Final DataFrame size: 24.48 MB


In [133]:
# using fetchall()
cursor = connection.cursor()
cursor.execute("SELECT * FROM Person.Person")
results = cursor.fetchall()
print(f"Memory usage after fetchall(): {get_memory_usage():.2f} MB")

# create dataframe from the results
df1 = pd.DataFrame.from_records(results, columns=[desc[0] for desc in cursor.description])
print("df1 memory", df1.memory_usage(deep = True).sum()/1024)
print(f"After creating DataFrame from fetchall() results: {get_memory_usage():.2f} MB")

# using pandas.read_sql()
df2 = pd.read_sql("SELECT * FROM Person.Person", connection)
print("df2 memory", df2.memory_usage(deep = True).sum())
print(f"Memory usage after read_sql(): {get_memory_usage():.2f} MB")

Memory usage after fetchall(): 1565.75 MB
df1 memory 20462.578125
After creating DataFrame from fetchall() results: 1568.20 MB


C:\Users\harun\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


df2 memory 20953680
Memory usage after read_sql(): 1581.30 MB


## 2. Using NumPy arrays vs Pandas DataFrames

A NumPy array is more memory-efficient than a pandas DataFrame. This is because a DataFrame has additional overhead due to its index and column label structures, as well as its ability to hold heterogeneous data types. A DataFrame essentially contains an underlying NumPy array, but also includes other data structures to support its extended functionality. Thus, if you have a large dataset composed of uniform data types and do not require the advanced functionalities provided by pandas, using a NumPy array could reduce your memory usage.

In the following script, the idea is to store the columns of the pandas dataframe as numpy arrays which reduces the memory usage drastically.

In [186]:
import copy
import time
#df = optimize(df)

# Create a dictionary to store numpy arrays
numpy_arrays = {}

start_time = time.time()

# Iterate over columns and create numpy arrays
for column in df.columns:
    # Note the additional list() wrapping
    numpy_arrays[column] = list([df[column].to_numpy()])

end_time = time.time()

print(f"Time spent creating numpy_arrays: {end_time - start_time} seconds \n")

# Print memory usage of each numpy array
for name, array in numpy_arrays.items():
    print(f"Memory usage of numpy array {name}: {getsizeof(array[0])} bytes")
    
start_time = time.time()

# Create a new DataFrame with numpy arrays at every column as single cells
df2 = pd.DataFrame(numpy_arrays)

end_time = time.time()
print(f"\nTime spent creating df2: {end_time - start_time} seconds")

start_time = time.time()

# Creating a new DataFrame from the numpy arrays, with the same dtypes of columns of df
df3 = pd.DataFrame({col: pd.Series(arr[0]) for col, arr in numpy_arrays.items()})

end_time = time.time()

print(f"\nTime spent creating df3: {end_time - start_time} seconds")

# Compare memory usage
print(f"\nMemory usage of original df: {df.memory_usage(deep=True).sum() / 1024**2} mbytes")
print(f"\nMemory usage of new df2: {df2.memory_usage(deep=True).sum() / 1024**2} mbytes")
print(f"\nMemory usage of new df3: {df3.memory_usage(deep=True).sum() / 1024**2} mbytes")

Time spent creating numpy_arrays: 0.03900456428527832 seconds
Memory usage of numpy array Unnamed: 0: 104 bytes
Memory usage of numpy array column_0_cat: 104 bytes
Memory usage of numpy array column_1_int: 104 bytes
Memory usage of numpy array column_2_float: 104 bytes
Memory usage of numpy array column_3_int: 104 bytes
Memory usage of numpy array column_4_float: 104 bytes
Memory usage of numpy array column_5_date: 40000104 bytes
Memory usage of numpy array column_6_int: 104 bytes
Memory usage of numpy array column_7_cat: 104 bytes
Memory usage of numpy array column_8_cat: 104 bytes
Memory usage of numpy array column_9_date: 104 bytes

Time spent creating df2: 0.007978677749633789 seconds

Time spent creating df3: 0.38344287872314453 seconds

Memory usage of original df: 1426.738751411438 mbytes

Memory usage of new df2: 38.14826965332031 mbytes

Memory usage of new df3: 1736.6516065597534 mbytes


Although a dataframe as numpy arrays consume little memory, since there is no straightforward way of perform pandas operations for data manipulation using numpy arrays, we need to create a dataframe from the numpy arrays again at some point. 

Converting data between pandas and NumPy involves overhead, both in terms of computational resources and in terms of code complexity. 

We can see that it takes a considerable amount of time when creating df3.

In [188]:
# Create DataFrame 1: 500000 rows, 1 column of integer type
df1 = pd.DataFrame({'col': range(5000000)})
df1_memory_usage = df1.memory_usage(deep=True).sum()
df1_sys_memory_usage = getsizeof(df1)
print(f"DataFrame 1 (5000000 rows, 1 column of integers):")
print(f"\tPandas memory_usage: {df1_memory_usage/1024} bytes")

# Create DataFrame 2: 5000000 rows, 1 column of single-element NumPy array
arr = np.array(range(5000000))
df2 = pd.DataFrame({'col': arr})
df2_memory_usage = df2.memory_usage(deep=True).sum()
df2_sys_memory_usage = getsizeof(df2)
print(f"\nDataFrame 2 (5000000 rows, 1 column of single-element numpy array):")
print(f"\tPandas memory_usage: {df2_memory_usage/1024} bytes")

# Create DataFrame 3: 1 row, 1 column with a NumPy array with 5000000 elements
df3 = pd.DataFrame({'col': [np.array(range(5000000))]})
df3_memory_usage = df3.memory_usage(deep=True).sum()
df3_sys_memory_usage = getsizeof(df3)
print(f"\nDataFrame 3 (1 row, 1 column of numpy array with 5000000 elements):")
print(f"\tPandas memory_usage: {df3_memory_usage/1024} bytes")

# Converting the dtype of the np array of df3 to int16
df3['col'] = df3['col'].apply(lambda x: x.astype('int16'))
df3_memory_usage = df3.memory_usage(deep=True).sum()
df3_sys_memory_usage = getsizeof(df3)
print(f"\nDataFrame 3 (1 row, 1 column of numpy array with 5000000 elements):")
print(f"\tPandas memory_usage: {df3_memory_usage/1024} bytes")

DataFrame 1 (5000000 rows, 1 column of integers):
	Pandas memory_usage: 39062.625 bytes

DataFrame 2 (5000000 rows, 1 column of single-element numpy array):
	Pandas memory_usage: 19531.375 bytes

DataFrame 3 (1 row, 1 column of numpy array with 5000000 elements):
	Pandas memory_usage: 19531.484375 bytes

DataFrame 3 (1 row, 1 column of numpy array with 5000000 elements):
	Pandas memory_usage: 9765.859375 bytes


## 3. Dask DataFrames vs Pandas DataFrames

Dask DataFrames are a large parallel DataFrame composed of smaller Pandas DataFrames. The large DataFrame is partitioned into several smaller chunks, where each chunk is a valid DataFrame itself. This allowes for distributed computation behind the scenes. Dask DataFrames support a large subset of the Pandas API, including groupbys, join operations, and sophisticated time series manipulations. Importantly, Dask operations are lazily evaluated, meaning computations are not executed until the result is explicitly requested. 

__Using Dask and Pandas Interchangeably:__ This can be a powerful strategy for dealing with memory limitations. The reason is Dask allows lazy evalutaion, which means computations are not performed until necessary, hence saving memory.  the ``compute()`` method is where all the computations take place. This can potentially save a lot of memory because data isn't loaded until necessary.

Whenever an operation that is not supported by Dask is required, the Dask DataFrame can be converted to a Pandas DataFrame. After performing the operation, the result can be converted back into a Dask DataFrame. This method leverages the strengths of both libraries, while avoiding memory overflow issues.

__Some fundamental Dask DataFrame arguments:__ The ``npartitions`` parameter specifies how many partitions you want to divide your Dask DataFrame into. For example, if you set ``npartitions=5``, your Dask DataFrame will consist of 5 smaller Pandas DataFrames. Bear in mind that having too few partitions could limit parallelism, having too many partitions can lead to slow task scheduling and increased memory usage.

In general, a good rule of thumb is to create partitions that are at least a few tens of megabytes in size, up to a maximum size that fits comfortably in memory. You might start with npartitions equal to twice the number of your machine's CPU cores and then adjust as necessary based on the memory usage and computation time.

Here is a script, comparing the time spent in operations where we retrieve data and perform merge:

In [214]:
import dask.dataframe as dd
import pandas as pd
import time
from sqlalchemy import create_engine

dask_total_time = 0

# SQL Server connection string
conn_str = (
    r'mssql+pyodbc:///?odbc_connect=' +
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=AdventureWorks2019;'
    r'Trusted_Connection=yes;'
)

engine = create_engine(conn_str)

# Load the data into pandas dataframes
start = time.time()
query1 = "SELECT * FROM Sales.SalesOrderDetail"
#df1 = pd.read_sql(query1, engine)
ddf1 = dd.from_pandas(pd.read_sql(query1, engine), npartitions=5)

query2 = "SELECT * FROM Sales.SalesOrderHeader"
#df2 = pd.read_sql(query2, engine)
ddf2 = dd.from_pandas(pd.read_sql(query2, cnxn), npartitions=5)
end = time.time()
print(f"Time taken to load data into pandas dataframes and convert to dask df: {end-start} seconds")
dask_total_time += (end-start)

# Merge operation in Dask
start = time.time()
merged_ddf = dd.merge(ddf1, ddf2, on='SalesOrderID', how='left')
end = time.time()
print(f"Time taken to merge dask dataframes: {end-start} seconds")
dask_total_time += (end-start)

# Convert merged dask dataframe back to pandas
start = time.time()
merged_df = merged_ddf.compute()
end = time.time()
print(f"Time taken to convert merged dask dataframe back to pandas: {end-start} seconds")
dask_total_time += (end-start)

print()
print(f"Total time taken in dask: {dask_total_time} seconds")
print()

# Convert merged dataframe to dask dataframe
#start = time.time()
#ddf_merged = dd.from_pandas(merged_df, npartitions=2)
#end = time.time()
#print(f"Time taken to convert merged pandas dataframe to dask: {end-start} seconds")

# Doing the same operation with pandas only
start = time.time()
query1 = "SELECT * FROM Sales.SalesOrderDetail"
df1_pandas = pd.read_sql(query1, engine)

query2 = "SELECT * FROM Sales.SalesOrderHeader"
df2_pandas = pd.read_sql(query2, engine)

merged_df_pandas = pd.merge(df1_pandas, df2_pandas, on='SalesOrderID', how='left')

end = time.time()
print(f"Time taken to load and merge dataframes using only pandas: {end-start} seconds")


C:\Users\harun\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Time taken to load data into pandas dataframes and convert to dask df: 2.1651906967163086 seconds
Time taken to merge dask dataframes: 0.029373884201049805 seconds
Time taken to convert merged dask dataframe back to pandas: 0.3315107822418213 seconds

Total time taken in dask: 2.5260753631591797 seconds

Time taken to load and merge dataframes using only pandas: 2.13199520111084 seconds


Using dask and pandas interchangeably is a little bit longer, but we load the data into memory as a whole only when it is necessary with ``compute()``. Reducing the probability of using too much memory during concurrent operations (say you have lots of requests coming in and memory usage of different operations add up and exceed the limit).

__Note that it could be faster if we load the data directly into a dask dataframe using ``read_sql_table``.__